**Занятие первое**

Начнем с простого. Многие знают что такое map и reduce операции, но все же для закрпеления мы их тут реализуем. Ах да, не забудем и про shuffle. Делать все будем на упрощенной задаче с word count для ознакомления с самим подходом.


На самом деле мы рассмптрим все в упрощенном виде, но это даст нам понимание, как можно через hadoop streaming, например, писать самописные map и reduce операции


! mapred streaming \
 -input /wiki/sample.jsonl \
 -output /word-count \
 -mapper "/opt/conda/bin/python3.6 mapper.py" \
 -reducer "/opt/conda/bin/python3.6 reducer.py" \
 -file mapper.py \
 -file reducer.py


Выше mapper.py и reducer.py это программы, которые выполняют одноименные операции нам потоком информации из jsonl файла, записывая ответ в файл word-count


In [1]:
import nltk


nltk.download("punkt")


nltk.download("stopwords")


from nltk.corpus import stopwords


import re


import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Petr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Petr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Давайте загрузим файл с текстом и посмотрим на него


In [54]:
with open("spark_text.txt", "rb") as f:
    data = f.readlines()
data = [text.decode() for text in data if text.decode() != "\r\n"]

In [55]:
len(data)

60

In [56]:
data[1]

'Apache Spark has its architectural foundation in the resilient distributed dataset (RDD), a read-only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way.[2] The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API. In Spark 1.x, the RDD was the primary application programming interface (API), but as of Spark 2.x use of the Dataset API is encouraged[3] even though the RDD API is not deprecated.[4][5] The RDD technology still underlies the Dataset API.[6][7]\r\n'

Как бы мы сделали..
Надо немного почистить слова, а также сделать все в парадигме MapReduce. Понятно, что можно все написать проще, но мы ведь хотим понять, как это работает=)

Загрузим стоп слова, очистим от них текст, приведем к нижнему регистру, всем раздадим ключи


In [6]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [ ]:
print(stop_words, sep="\t")

{"they're", 'who', "hadn't", "wasn't", 'all', "you've", 'himself', 'what', 'those', 'theirs', 'me', 'same', 'each', 'until', 'by', 'other', 'my', 'she', "isn't", 'your', 'isn', 'on', 'can', 'whom', 'doing', 'through', 'but', 'it', 'off', "that'll", 'during', 'him', 'or', 'how', 'weren', "won't", 'herself', 'only', "she's", 'hers', 'these', 'o', 'ma', 'at', 'should', 'both', 'won', 'm', "he'll", 'their', 'myself', "should've", 'her', 'for', 'and', 'from', 'his', 'where', 'aren', 'between', 'didn', 'nor', 'over', 'so', 'this', 'wasn', 'any', 'too', 'shan', "i'll", "i've", 'do', "you'll", "they'll", 'had', "we've", 'out', 'most', "you're", 'few', "haven't", "we'll", "hasn't", 'he', 'under', 'more', "i'd", 'about', 'mustn', 'have', 'its', 'against', 'a', "shouldn't", "he'd", 'such', 'hasn', 'did', 'doesn', 'after', "wouldn't", "they'd", 'yours', 'here', "mightn't", 'you', "don't", 'because', 'don', 'some', "it'd", 've', 'in', 'up', 'yourself', 'haven', 'with', 'd', "didn't", 'further', 'ju

пунктуацию тоже полезно бы удалить


In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words if word not in stop_words]
    words_with_value = sorted(words_with_value, key=lambda x: x[0])
    return words_with_value


def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())


def shuffle_text(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key) % n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks


# на самом деле для reduce в жизни пишут иначе..не зря мы сортируем внутри map
# данные по ключам. Это нужно для избавления от этапа проверки ключа и поиска
def reduce_text(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

Проверим, что все работает

Сначала map


In [57]:
data[0]

'Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\r\n'

In [58]:
map_stage = mapper_text(data[0])

In [15]:
map_stage

[('amplab', 1),
 ('analytics', 1),
 ('apache', 1),
 ('apache', 1),
 ('berkeley', 1),
 ('california', 1),
 ('clusters', 1),
 ('codebase', 1),
 ('data', 1),
 ('data', 1),
 ('developed', 1),
 ('donated', 1),
 ('engine', 1),
 ('entire', 1),
 ('fault', 1),
 ('foundation', 1),
 ('implicit', 1),
 ('interface', 1),
 ('largescale', 1),
 ('later', 1),
 ('maintained', 1),
 ('opensource', 1),
 ('originally', 1),
 ('parallelism', 1),
 ('processing', 1),
 ('programming', 1),
 ('provides', 1),
 ('since', 1),
 ('software', 1),
 ('spark', 1),
 ('spark', 1),
 ('spark', 1),
 ('tolerance', 1),
 ('unified', 1),
 ('university', 1)]

shuffle


In [59]:
shuffle_stage = shuffle_text(map_stage, 5)

[(0, ('amplab', 1)), (0, ('apache', 1)), (0, ('apache', 1)), (0, ('berkeley', 1)), (0, ('donated', 1)), (0, ('engine', 1)), (0, ('entire', 1)), (0, ('implicit', 1)), (0, ('opensource', 1)), (0, ('processing', 1)), (0, ('provides', 1)), (1, ('clusters', 1)), (1, ('codebase', 1)), (1, ('maintained', 1)), (1, ('originally', 1)), (1, ('parallelism', 1)), (1, ('spark', 1)), (1, ('spark', 1)), (1, ('spark', 1)), (1, ('university', 1)), (2, ('analytics', 1)), (2, ('data', 1)), (2, ('data', 1)), (2, ('interface', 1)), (2, ('later', 1)), (2, ('software', 1)), (2, ('tolerance', 1)), (3, ('fault', 1)), (3, ('programming', 1)), (3, ('since', 1)), (4, ('california', 1)), (4, ('developed', 1)), (4, ('foundation', 1)), (4, ('largescale', 1)), (4, ('unified', 1))]


In [50]:
shuffle_stage

{0: [('clusters', 1),
  ('donated', 1),
  ('implicit', 1),
  ('interface', 1),
  ('largescale', 1),
  ('later', 1),
  ('opensource', 1),
  ('university', 1),
  ('1x', 1),
  ('deprecated45', 1),
  ('faulttolerant', 1),
  ('interface', 1),
  ('items', 1),
  ('across', 1),
  ('cluster', 1),
  ('linear', 1),
  ('mapreduce', 1),
  ('mapreduce', 1),
  ('reduce', 1),
  ('form', 1),
  ('analysis', 1),
  ('ms', 1),
  ('among', 1),
  ('impetus', 1),
  ('initial', 1),
  ('mapreduce', 1),
  ('either', 1),
  ('install', 1),
  ('launch', 1),
  ('launch', 1),
  ('management', 1),
  ('possible', 1),
  ('storage', 1),
  ('amazon', 1),
  ('interface', 1),
  ('purposes', 1),
  ('s3', 1),
  ('testing', 1),
  ('application', 1),
  ('available', 1),
  ('interface', 1),
  ('julia17', 1),
  ('scheduling', 1),
  ('cluster2', 1),
  ('data', 1),
  ('interface', 1),
  ('new', 1),
  ('parallel', 1),
  ('parallel', 1),
  ('reconstructed', 1),
  ('take', 1),
  ('type', 1),
  ('available', 1),
  ('data', 1),
  ('impe

reduce


In [18]:
reduce_text(shuffle_stage[4][1])

{'california': 1,
 'developed': 1,
 'foundation': 1,
 'largescale': 1,
 'unified': 1}

Итак, осталось все рассчитать параллельно и собрать результаты


In [9]:
from joblib import Parallel, delayed

In [20]:
n_nodes = 5

Обернем в 1 функциию для удобства map и shuffle


In [21]:
def map_shuffle(text, n_nodes):
    map_result = mapper_text(text)
    shuffle_result = shuffle_text(map_result, n_nodes)
    return shuffle_result

In [22]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle)(df, n_nodes) for df in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed:    1.5s finished


In [23]:
len(res)

60

In [24]:
res[0]

[(0,
  [('clusters', 1),
   ('donated', 1),
   ('implicit', 1),
   ('interface', 1),
   ('largescale', 1),
   ('later', 1),
   ('opensource', 1),
   ('university', 1)]),
 (1,
  [('analytics', 1),
   ('california', 1),
   ('processing', 1),
   ('programming', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('tolerance', 1)]),
 (2,
  [('amplab', 1),
   ('codebase', 1),
   ('data', 1),
   ('data', 1),
   ('developed', 1),
   ('fault', 1)]),
 (3,
  [('apache', 1),
   ('apache', 1),
   ('berkeley', 1),
   ('engine', 1),
   ('entire', 1),
   ('foundation', 1),
   ('maintained', 1),
   ('originally', 1),
   ('parallelism', 1),
   ('since', 1),
   ('software', 1),
   ('unified', 1)]),
 (4, [('provides', 1)])]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова


In [25]:
shuffle_stage = {i: [] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [26]:
for key in shuffle_stage.keys():
    print(f"{key}: number of words = {len(shuffle_stage[key])}")

0: number of words = 322
1: number of words = 399
2: number of words = 436
3: number of words = 407
4: number of words = 357


И последний этап - нужно сделать reduce


In [27]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(
        delayed(reduce_text)(shuffle_stage[key]) for key in shuffle_stage.keys()
    )

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [28]:
len(res)

5

In [29]:
res[0]

{'clusters': 1,
 'donated': 2,
 'implicit': 1,
 'interface': 8,
 'largescale': 1,
 'later': 2,
 'opensource': 1,
 'university': 1,
 '1x': 1,
 'deprecated45': 1,
 'faulttolerant': 1,
 'items': 1,
 'across': 2,
 'cluster': 7,
 'linear': 1,
 'mapreduce': 7,
 'reduce': 1,
 'form': 2,
 'analysis': 2,
 'ms': 1,
 'among': 1,
 'impetus': 1,
 'initial': 1,
 'either': 1,
 'install': 1,
 'launch': 2,
 'management': 1,
 'possible': 1,
 'storage': 1,
 'amazon': 1,
 'purposes': 1,
 's3': 1,
 'testing': 1,
 'application': 2,
 'available': 2,
 'julia17': 1,
 'scheduling': 2,
 'cluster2': 1,
 'data': 11,
 'new': 2,
 'parallel': 2,
 'reconstructed': 1,
 'take': 1,
 'type': 1,
 'imperative': 1,
 'needs': 1,
 'restricted': 1,
 'two': 1,
 'variables': 2,
 'example': 1,
 'occurring': 1,
 'performs': 2,
 'simple': 2,
 'takes': 1,
 'typical': 1,
 'words': 2,
 'called': 2,
 'fully': 1,
 'interfaces': 1,
 'introduced': 1,
 'lack': 1,
 'language': 2,
 'typechecking': 1,
 'well': 2,
 'enables': 1,
 'perform': 1,


Собираем результат


In [30]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [31]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('spark', 110),
 ('apache', 56),
 ('data', 44),
 ('streaming', 36),
 ('distributed', 23),
 ('processing', 19),
 ('sql', 17),
 ('also', 15),
 ('learning', 15),
 ('rdd', 15),
 ('api', 14),
 ('structured', 13),
 ('cluster', 12),
 ('machine', 12),
 ('hadoop', 11),
 ('interface', 10),
 ('mapreduce', 10),
 ('rdds', 10),
 ('provides', 10),
 ('core', 9),
 ('framework', 9),
 ('graph', 9),
 ('programming', 9),
 ('the', 9),
 ('use', 9),
 ('mllib', 9),
 ('application', 8),
 ('used', 8),
 ('support', 8),
 ('tasks', 8),
 ('code', 8),
 ('python', 8),
 ('operations', 7),
 ('it', 7),
 ('dataset', 7),
 ('in', 7),
 ('algorithms', 7),
 ('graphx', 7),
 ('run', 7),
 ('deep', 7),
 ('two', 6),
 ('simple', 6),
 ('including', 6),
 ('developers', 6),
 ('scala', 6),
 ('implementation', 6),
 ('batch', 6),
 ('pipelines', 6),
 ('graphs', 6),
 ('map', 6),
 ('big', 6),
 ('abstraction', 6),
 ('across', 5),
 ('new', 5),
 ('parallel', 5),
 ('top', 5),
 ('supports', 5),
 ('analytics', 5),
 ('even', 5),
 ('applications', 

Да, было бы проще все сделать иным кодом и в один проход, но целью было разобрать, как все это примерно работает под капотом на больших данных.


**Домашнее задание**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок


In [13]:
from joblib import Parallel, delayed
from collections import Counter

In [1]:
with open("user_ratedmovies.dat", "rb") as f:
    data = f.readlines()
headers = data[0].decode().split("\t")[:3]
data = [row.decode().split("\t")[:3] for row in data[1:]]

In [2]:
headers

['userID', 'movieID', 'rating']

In [3]:
data[0]

['75', '3', '1']

In [4]:
len(data)

855598

Пишем map, shiffle и reduce + параллелим вычисления. Лучше задавать batch_size при распараллеливании, либо даже заранее все разбить на батчи, будет быстрее

Также посмотрите на то, нет ли перекоса в данных после shuffle, можете попробовать использовать остаток от деления не простого hash, а ввести какую-то функию


In [7]:
def map_rating(row):
    rating = float(row[2])
    if rating > 4.0:
        return [(row[1], 1)]
    return []


def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())


def shuffle_rating(mapper_result, n_nodes=5):
    shuffled = [(hash(k) % n_nodes, (k, v)) for k, v in mapper_result]
    shuffled.sort(key=lambda x: x[0])
    return create_chunks(shuffled)


def reduce_rating(map_row):
    result = {}
    for idx, num in map_row:
        if idx in result:
            result[idx] += 1
        else:
            result[idx] = num
    return result


def map_shuffle(batch, n_nodes):
    mapped = []
    for row in batch:
        mapped.extend(map_rating(row))
    return shuffle_rating(mapped, n_nodes)

In [ ]:
n_nodes = 5
BATCH = 10**4


def make_batches(data, batch_size=BATCH):
    for i in range(0, len(data), batch_size):
        yield data[i : i + batch_size]

In [11]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=BATCH) as parallel:
    res = parallel(delayed(map_shuffle)(batch, n_nodes) for batch in make_batches(data))

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done  13 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done  41 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done  50 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done  61 tasks      | elapsed:    0.6s
[Parallel(n_jobs=5)]: Done  72 tasks      | elapsed:    0.6s
[Parallel(n_jobs=5)]: Done  86 out of  86 | elapsed:    0.7s finished


In [15]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    reduced_parts = parallel(
        delayed(reduce_rating)(shuffle_stage[pid]) for pid in range(n_nodes)
    )

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.2s finished


In [32]:
total = Counter()
for part_dict in reduced_parts:
    total.update(part_dict)

top_movies = sorted(total.items(), key=lambda kv: kv[1], reverse=True)
print("Номер фильма | Количество просмотров")
for movie_id, cnt in top_movies[:5]:
    print(f"{movie_id}\t\t\t{cnt}")

Номер фильма | Количество просмотров
2571			900
318			887
296			878
2959			828
4993			756


После reduce все можно собрать в одном цикле, считаем, что данные переслали после на 1 машину и агрегируем
